In [24]:
import asyncio
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv
import os

load_dotenv()
api_key= os.getenv("OPENAI_API_KEY")
model_client = OpenAIChatCompletionClient(model="gpt-4o", api_key=api_key)

python-dotenv could not parse statement starting at line 5
python-dotenv could not parse statement starting at line 6


In [25]:
from autogen_agentchat.agents import AssistantAgent

teacher = AssistantAgent(
    name="Teacher",
    model_client=model_client,
    description="Explains a topic step by step",
    system_message="You are a patient teacher. Explain concepts clearly and step by step."
)

quiz_master = AssistantAgent(
    name="QuizMaster",
    model_client=model_client,
    description="Generates practice questions",
    system_message="You create 3-5 practice questions based on the Teacher's explanation."
)

grader = AssistantAgent(
    name="Grader",
    model_client=model_client,
    description="Checks answers and provides feedback",
    system_message="You grade the learner's answers fairly. Keep feedback under 50 words."
)

motivator = AssistantAgent(
    name="Motivator",
    model_client=model_client,
    description="Encourages the learner",
    system_message="You provide encouragement, tips, and motivation. Be supportive and brief.Say 'TERMINATE' when the learner seems confident."
)


In [26]:
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination

termination_condition = TextMentionTermination(text="TERMINATE") 

edu_team = RoundRobinGroupChat(
    participants=[teacher, quiz_master, grader, motivator],
    termination_condition=termination_condition,
    max_turns=5
)

In [27]:
from autogen_agentchat.messages import TextMessage
import asyncio

async def run_team():
    task = TextMessage(
        content="Explain about RAG Pipeline.",
        source="User"
    )

    result = await edu_team.run(task=task)

    for msg in result.messages:
        print(f"{msg.source}: {msg.content}")

await run_team()

User: Explain about RAG Pipeline.
Teacher: Certainly! The RAG (Retrieval-Augmented Generation) pipeline is a framework that combines information retrieval and text generation. It is particularly useful for tasks that require producing or generating informative text-based responses by leveraging large datasets or documents. Let's break it down step-by-step:

### 1. Understanding the Components:

#### a. Retrieval:
- **Purpose:** To find relevant documents or pieces of information from a large dataset or corpus that can potentially aid in generating a more informed and accurate response.
- **Method:** Typically done using a retriever model, which could be a traditional Information Retrieval (IR) method (like TF-IDF or BM25) or more advanced methods using neural networks (like Dense Passage Retrieval - DPR).
- **Output:** The retrieval step provides a subset of documents or passages deemed highly relevant to the query or input.

#### b. Generation:
- **Purpose:** To synthesize a coherent 